In [1]:
import numpy as np
import pandas as pd

from dataset_processing import split_dataset, get_starting_dataset, enrich_dataset, get_movie_map
from similarity import pearson_similarity
from predict import prediction_function
from group import alpha_hybrid_aggregation, update_alpha_mom_disagreements
from evaluate import user_satisfaction

from datetime import datetime

In [2]:
NUM_ITERATIONS = 5
MAX_NEIGHBORS = 75      # ~ 3*np.sqrt(num_users)
MAX_RECOMMENDATIONS = 10

In [3]:
movie_map = get_movie_map()

In [23]:
group = [437, 329, 73]

In [36]:
it_counter = 1
alpha = 0
df_user_sat = pd.DataFrame(index=group, columns=["It1", "It2", "It3", "It4", "It5"])
df_user_satO = pd.DataFrame(index=group, columns=["It1", "It2", "It3", "It4", "It5"])
group_disagreements = list()

In [37]:
while it_counter < 6:
    print(f"ITERATION: {it_counter}" + " -> ALPHA = " + str(alpha) + "\n")
    start = datetime.now()

    if it_counter == 1:
        first_half, chunks = split_dataset(NUM_ITERATIONS, 111)
        matrix = get_starting_dataset(first_half)
    else:
        matrix = enrich_dataset(matrix, chunks[it_counter-2])

    # Compute Similarities
    print("First Step: Compute Similarities of each User of the Group")
    list_of_similarities = list()
    for user in group:
        dictionary = dict()

        other_users = [u for u in matrix.index if u != user]
        for u in other_users:
            dictionary[u] = pearson_similarity(matrix, user, u)
        list_of_similarities.append(dictionary)
    
    # Neighborhood (75) Building
    print("Second Step: Neighborhood Building")
    for i in range(0, len(list_of_similarities)):
        similarities = list_of_similarities.pop(i)
        similarities = {k: v for k, v in sorted(similarities.items(), key=lambda item: item[1], reverse=True)}
        similarities = dict(list(similarities.items())[:MAX_NEIGHBORS])
        list_of_similarities.insert(i, similarities)
    
    # Predictions for Each User
    print("Third Step: Predictions for Each User")
    list_of_scores = list()

    for i in range(0, len(group)):
        scores = prediction_function(matrix, group[i], list_of_similarities[i], matrix.shape[1])
        list_of_scores.append(scores)
    
    # Group Recommendation
    print("Fourth Step: Group Recommendation")
    score_matrix = alpha_hybrid_aggregation(alpha, list_of_scores)

    # Evaluation: Group Satisfaction & Group Disagreements
    print("Fifth Step: Evaluation")
    for i, user in enumerate(group):
        sat = user_satisfaction(score_matrix, list_of_scores[i])
        df_user_sat.at[user, f"It{it_counter}"] = sat
        df_user_satO.at[user, f"It{it_counter}"] = np.mean(df_user_sat.loc[user])
        # print(f"User {user} -> Satisfaction: {sat:.5f}")
    
    group_satO = np.sum(df_user_satO[f'It{it_counter}'])/3
    print("Group Satisfaction Overall: " + str(group_satO))

    group_dis = np.max(df_user_satO[f'It{it_counter}']) - np.min(df_user_satO[f'It{it_counter}'])
    group_disagreements.append(group_dis)
    print("Group Disagreement Overall: " + str(group_dis))

    # Getting ready for the Next Iteration
    alpha = update_alpha_mom_disagreements(df_user_sat[f"It{it_counter}"])

    end = datetime.now()
    print(f"END OF ITERATION {it_counter} - Elapsed Time: {end-start}" + "\n")

    it_counter += 1

ITERATION: 1 -> ALPHA = 0

First Step: Compute Similarities of each User of the Group
Second Step: Neighborhood Building
Third Step: Predictions for Each User
Fourth Step: Group Recommendation
Fifth Step: Evaluation
Group Satisfaction Overall: 0.8329376802148646
Group Disagreement Overall: 0.04665310542946355
END OF ITERATION 1 - Elapsed Time: 0:00:33.816977

ITERATION: 2 -> ALPHA = 0.027010131093429368

First Step: Compute Similarities of each User of the Group
Second Step: Neighborhood Building
Third Step: Predictions for Each User
Fourth Step: Group Recommendation
Fifth Step: Evaluation
Group Satisfaction Overall: 0.8289460914022871
Group Disagreement Overall: 0.057757190419043924
END OF ITERATION 2 - Elapsed Time: 0:00:28.737165

ITERATION: 3 -> ALPHA = 0.05355118170005019

First Step: Compute Similarities of each User of the Group
Second Step: Neighborhood Building
Third Step: Predictions for Each User
Fourth Step: Group Recommendation
Fifth Step: Evaluation
Group Satisfaction Ove

# Risultati Prima Esecuzione

In [26]:
df_user_sat

,It1,It2,It3,It4,It5
437,0.895671,0.923058,0.906435,0.868639,0.926059
329,0.710293,0.772807,0.686127,0.736088,0.731574
73,0.861083,0.813863,0.896151,0.833515,0.717571


In [27]:
# seed == 24
df_user_satO

,It1,It2,It3,It4,It5
437,0.895671,0.909365,0.908388,0.898451,0.903972
329,0.710293,0.74155,0.723075,0.726329,0.727378
73,0.861083,0.837473,0.857032,0.851153,0.824436


In [28]:
df_user_satO.mean()

It1    0.822349
It2    0.829462
It3    0.829499
It4    0.825311
It5    0.818595
dtype: object

In [29]:
group_disagreements

[0.18537856360645688,
 0.16781518356366798,
 0.18531266768042765,
 0.17212209816006685,
 0.17659461635819917]

# Risultati Seconda Esecuzione

In [32]:
df_user_sat

,It1,It2,It3,It4,It5
437,0.862811,0.863441,0.818811,0.876081,0.926059
329,0.782876,0.738604,0.761684,0.749251,0.731574
73,0.88638,0.77201,0.77521,0.772559,0.717571


In [33]:
# seed == 8
df_user_satO

,It1,It2,It3,It4,It5
437,0.862811,0.863126,0.848354,0.855286,0.869441
329,0.782876,0.76074,0.761055,0.758104,0.752798
73,0.88638,0.829195,0.8112,0.80154,0.784746


In [34]:
df_user_satO.mean()

It1    0.844022
It2    0.817687
It3     0.80687
It4    0.804976
It5    0.802328
dtype: object

In [35]:
group_disagreements

[0.10350401338122173,
 0.10238620369111129,
 0.08729973061039165,
 0.09718223532520864,
 0.11664272609031268]

# Risultati Terza Esecuzione

In [38]:
df_user_sat

,It1,It2,It3,It4,It5
437,0.850944,0.789254,0.908204,0.90746,0.926059
329,0.804291,0.804691,0.753483,0.71455,0.731574
73,0.843577,0.880919,0.767288,0.755261,0.717571


In [39]:
# seed == 111
df_user_satO

,It1,It2,It3,It4,It5
437,0.850944,0.820099,0.849467,0.863965,0.876384
329,0.804291,0.804491,0.787488,0.769254,0.761718
73,0.843577,0.862248,0.830595,0.811761,0.792923


In [40]:
df_user_satO.mean()

It1    0.832938
It2    0.828946
It3    0.822517
It4    0.814994
It5    0.810342
dtype: object

In [41]:
group_disagreements

[0.04665310542946355,
 0.057757190419043924,
 0.06197881459099597,
 0.09471166107776341,
 0.11466626669235636]